# <span style="font-width:bold; font-size: 3rem; color:#1EB182;">**Hopsworks Feature Store** </span> <span style="font-width:bold; font-size: 3rem; color:#333;">- Part 05: Batch Predictions</span>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/electricity/5_batch_predictions.ipynb)

## 🗒️ This notebook is divided into 2 main sections:

1. **Loading batch data**,
2. **Prediction using model from Model Registry**.

![tutorial-flow](https://github.com/logicalclocks/hopsworks-tutorials/blob/master/images/03_model.png?raw=1)

### <span style="color:#ff5f27;">📝 Importing Libraries</span>

In [ ]:
!pip install -U hopsworks --quiet
!pip install -U tensorflow --quiet

In [ ]:
import tensorflow as tf
import pandas as pd

---

## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

---

## <span style="color:#ff5f27;"> ✨ Load recent batch data</span>

First, you will need to fetch the training dataset that you created in the previous notebook. You will use January-February data.

In [ ]:
import datetime
start_date = (datetime.datetime.now() - datetime.timedelta(days=5)) 
start_time = int(start_date.timestamp()) * 1000

In [ ]:
feature_view = fs.get_feature_view("electricity_feature_view", 1)
feature_view.init_batch_scoring(training_dataset_version=1)
df = feature_view.get_batch_data(start_time=start_time)
df.sort_values(["timestamp"], inplace=True)

In [ ]:
df = df.drop(["day", "timestamp"], axis=1)

In [ ]:
df

---

## <span style="color:#ff5f27;">🚀 Using the model to predict electricity prices</span>

### <span style='color:#ff5f27'>🗄 Retrieving model from Model Registry</span>

In [ ]:
mr = project.get_model_registry()
model = mr.get_model("electricity_price_prediction_model", version=1)
model_dir = model.download()
loaded_model = tf.saved_model.load(model_dir)
serving_function = loaded_model.signatures["serving_default"]

### <span style='color:#ff5f27'>🤖 Making the predictions</span>

In [ ]:
z = serving_function(tf.constant(df.values.reshape(-1, df.shape[0], 29), tf.float32))
prediction = z[list(z.keys())[0]].numpy() #.flatten().tolist()
prediction

In [ ]:
price_se1_pred = []
price_se2_pred = []
price_se3_pred = []
price_se4_pred = []

for batch_n in range(len(prediction)):
    batch = prediction[batch_n]
    for window_n in range(df.shape[0]):
        price_se1_pred.append(batch[window_n][0])
        price_se2_pred.append(batch[window_n][1])
        price_se3_pred.append(batch[window_n][2])
        price_se4_pred.append(batch[window_n][3])

In [ ]:
feature_view.init_serving(1)

In [ ]:
# td_transformation_functions = feature_view._single_vector_server._transformation_function

td_transformation_functions = feature_view._batch_scoring_server._transformation_functions

In [ ]:
# decoding our features

preds = pd.DataFrame(data={'price_se1': price_se1_pred,
                           'price_se2': price_se2_pred,
                           'price_se3': price_se3_pred,
                           'price_se4': price_se4_pred,})
import inspect 

res = {}
for feature_name in td_transformation_functions:
    if feature_name in ["price_se1", "price_se2", "price_se3", "price_se4"]:
        td_transformation_function = td_transformation_functions[feature_name]
        sig, foobar_locals = inspect.signature(td_transformation_function.transformation_fn), locals()
        param_dict = dict([(param.name, param.default) for param in sig.parameters.values() if param.default != inspect._empty])
        if td_transformation_function.name == "min_max_scaler":
            preds[feature_name] = preds[feature_name].map(
                lambda x: x * (param_dict["max_value"] - param_dict["min_value"]) + param_dict["min_value"]
                )
            
preds = preds.apply(lambda x: -x)

preds

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

%config InlineBackend.figure_format='retina'
%matplotlib inline

plt.plot(preds["price_se1"],color='red', label='SE1 price prediction')
plt.plot(preds["price_se2"], color='blue', label='SE2 price prediction')
plt.plot(preds["price_se3"], color='green', label='SE3 price prediction')
plt.plot(preds["price_se4"], color='black', label='SE4 price prediction')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend(loc='upper left')

#save figure to PNG file
plt.savefig('model_preds.png', dpi=300)

fig = plt.gcf()
fig.set_size_inches(15, 5)